In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Cropping1D, add, Flatten, GlobalAvgPool1D, Dense

# Define a minimal version of the model
def recreate_model(sequence_len, out_pred_len, filters=64, n_dil_layers=4, conv1_kernel_size=21, profile_kernel_size=75, num_tasks=1):
    # Input layer
    inp = Input(shape=(sequence_len, 4), name='sequence')    

    # First convolution
    x = Conv1D(filters,
               kernel_size=conv1_kernel_size,
               padding='valid', 
               activation='relu',
               name='conv1')(inp)

    # Dilated convolutions with cropping
    for i in range(1, n_dil_layers + 1):
        conv_x = Conv1D(filters, 
                        kernel_size=3, 
                        padding='valid',
                        activation='relu', 
                        dilation_rate=2**i,
                        name=f'dilated_conv_{i}')(x)

        x_len = x.shape[1]
        conv_x_len = conv_x.shape[1]
        crop_size = (x_len - conv_x_len) // 2
        x = Cropping1D(crop_size, name=f'crop_{i}')(x)
        x = add([conv_x, x])

    # Profile prediction branch
    prof_out_precrop = Conv1D(filters=num_tasks,
                              kernel_size=profile_kernel_size,
                              padding='valid',
                              name='profile_conv')(x)

    cropsize = int(prof_out_precrop.shape[1] / 2) - int(out_pred_len / 2)
    prof = Cropping1D(cropsize, name='profile_crop')(prof_out_precrop)
    profile_out = Flatten(name="profile_output")(prof)

    # Counts prediction branch
    gap_combined_conv = GlobalAvgPool1D(name='gap')(x)
    count_out = Dense(num_tasks, name="count_output")(gap_combined_conv)

    # Build model
    model = Model(inputs=[inp], outputs=[profile_out, count_out])
    return model


In [3]:

# Example parameters
sequence_len = 1000
out_pred_len = 500
filters = 64
n_dil_layers = 4

# Create and summarize the model
model = recreate_model(sequence_len, out_pred_len, filters=filters, n_dil_layers=n_dil_layers)
model.summary()

# Dummy data to check shapes
import numpy as np
dummy_input = np.random.random((1, sequence_len, 4)).astype(np.float32)
profile_out, count_out = model.predict(dummy_input)

# Print shapes
print("Input shape:", dummy_input.shape)
print("Profile output shape:", profile_out.shape)
print("Count output shape:", count_out.shape)


2024-11-18 16:46:13.349757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-18 16:46:13.350290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-11-18 16:46:13.350835: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-11-18 16:46:13.351362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2024-11-18 16:46:13.351823: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1000, 4)]    0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, 980, 64)      5440        ['sequence[0][0]']               
                                                                                                  
 dilated_conv_1 (Conv1D)        (None, 976, 64)      12352       ['conv1[0][0]']                  
                                                                                                  
 crop_1 (Cropping1D)            (None, 976, 64)      0           ['conv1[0][0]']                  
                                                                                              

In [7]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# PyTorch CNN module and Flatten module
class CNNModule(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, activation_fn=nn.ReLU):
        super().__init__()
        self.conv = nn.Conv1d(in_channels=in_channels,
                            out_channels=out_channels, 
                            kernel_size=kernel_size, 
                            stride=stride, 
                            padding=padding)
        self.activation = activation_fn()
    
    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.flatten(x)
        return x
    
class Cropping1D(nn.Module):
    def __init__(self, crop_size):
        super().__init__()
        self.crop_size = crop_size  # Total amount to crop (both sides combined)

    def forward(self, x):
        # For skip connections, we need to crop from both ends equally
        crop_left = self.crop_size // 2
        crop_right = self.crop_size - crop_left
        
        if crop_right > 0:
            return x[:, :, crop_left:-crop_right]
        return x[:, :, crop_left:]

batch_size = 8
channels = 4
seq_length = 1000

output_length=500
input_tensor = torch.randn(batch_size, channels, seq_length)  # (batch_size, channels, sequence_length)

# PyTorch model pipeline
cnn_module = CNNModule(in_channels=channels, out_channels=1, kernel_size=3, stride=1, padding=0)
flatten_module = Flatten()

cropping_module = Cropping1D(1+(seq_length-output_length))

# Passing through CNN and Flatten in PyTorch
cnn_output = cnn_module(input_tensor)
cnn_output_shape = cnn_output.shape
cropping_output = cropping_module(cnn_output)
cropping_output_shape = cropping_output.shape
flatten_output = flatten_module(cropping_output)
flatten_output_shape = flatten_output.shape


# TensorFlow model equivalent
input_tensor_tf = tf.random.normal([batch_size, seq_length, channels])  # (batch_size, sequence_length, channels)

# TensorFlow CNN layer
conv_layer_tf = tf.keras.layers.Conv1D(filters=1, kernel_size=3, strides=1, padding='valid', activation='relu')
conv_output_tf = conv_layer_tf(input_tensor_tf)
conv_output_shape_tf = conv_output_tf.shape

# TensorFlow Cropping layer (Cropping1D equivalent)
crop_layer_tf = tf.keras.layers.Cropping1D(cropping=(1 + (seq_length - output_length) // 2 , 1 +(seq_length - output_length) // 2 +1))
crop_output_tf = crop_layer_tf(conv_output_tf)
crop_output_tf_shape = crop_output_tf.shape

# TensorFlow Flatten layer
flatten_layer_tf = tf.keras.layers.Flatten()
flatten_output_tf = flatten_layer_tf(crop_output_tf)
flatten_output_shape_tf = flatten_output_tf.shape

# Print shapes for comparison
print(f"PyTorch CNN output shape: {cnn_output_shape}")
print(f"PyTorch Flatten output shape: {flatten_output_shape}")
print(f"PyTorch Crop output shape: {cropping_output_shape}")

print(f"TensorFlow CNN output shape: {conv_output_shape_tf}")
print(f"TensorFlow Flatten output shape: {flatten_output_shape_tf}")
print(f"TensorFlow Crop output shape: {crop_output_tf_shape}")


PyTorch CNN output shape: torch.Size([8, 1, 998])
PyTorch Flatten output shape: torch.Size([8, 497])
PyTorch Crop output shape: torch.Size([8, 1, 497])
TensorFlow CNN output shape: (8, 998, 1)
TensorFlow Flatten output shape: (8, 495)
TensorFlow Crop output shape: (8, 495, 1)


In [18]:
import torch.nn.functional as F

class MultinomialNLLLoss(nn.Module):
    def __init__(self):
        """
        Multinomial Negative Log-Likelihood Loss.
        
        Computes the negative log-likelihood for multinomial distribution.
        """
        super().__init__()
    
    def forward(self, logits, true_counts):
        """
        Compute the multinomial negative log-likelihood.
        
        Args:
            logits (torch.Tensor): Predicted logit values 
                Shape: (batch_size, num_categories)
            true_counts (torch.Tensor): Observed count values 
                Shape: (batch_size, num_categories)
        
        Returns:
            torch.Tensor: Average negative log-likelihood across the batch
        """
        # Compute log probabilities for each category
        log_probs = F.log_softmax(logits, dim=-1)
        
        # Compute total counts per example
        total_counts = true_counts.sum(dim=-1)
        
        # Compute the log-likelihood manually
        # This is equivalent to the multinomial distribution log probability
        likelihood = torch.sum(true_counts * log_probs, dim=-1)
        
        # Apply constraint that probabilities sum to 1 and match total count
        # This is similar to the multinomial distribution constraint
        log_likelihood = likelihood - torch.lgamma(total_counts + 1)
        
        # Return negative mean log-likelihood
        return -log_likelihood.mean()

In [21]:
# Create the loss
loss_fn = MultinomialNLLLoss()

# Assume you have your logits and true counts
logits = torch.randn(32, 1000)  # 32 examples, 10 categories
true_counts = torch.randint(0, 10, (32, 1000)).float()

# Compute loss
loss = loss_fn(logits, true_counts)

In [22]:
loss

tensor(66587.3594)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from typing import Tuple, List

class ChromBPNetTest(nn.Module):
    def __init__(self, 
                sequence_len: int = 1000,
                out_pred_len: int = 1000,
                filters: int = 64,
                n_dil_layers: int = 8,
                conv1_kernel_size: int = 21,
                profile_kernel_size: int = 75):
        super().__init__()
        
        print("\nInitializing ChromBPNet Test Network")
        print(f"Input sequence length: {sequence_len}")
        print(f"Target output length: {out_pred_len}")
        print(f"Number of filters: {filters}")
        print(f"Number of dilated layers: {n_dil_layers}")
        print(f"First conv kernel size: {conv1_kernel_size}")
        print(f"Profile conv kernel size: {profile_kernel_size}\n")
        
        # Track shapes at each layer
        self.layer_shapes = {}
        
        # Initial conv with no dilation
        self.initial_conv = nn.Conv1d(4, filters, kernel_size=conv1_kernel_size, padding=0)
        
        # Dilated convolution layers
        self.dilated_convs = nn.ModuleList()
        for i in range(1, n_dil_layers + 1):
            dilation = 2 ** i
            self.dilated_convs.append(
                nn.Conv1d(filters, filters, kernel_size=3, 
                         padding=0, dilation=dilation)
            )
        
        # Profile prediction branch
        self.profile_conv = nn.Conv1d(filters, 1, kernel_size=profile_kernel_size, padding=0)
        
        # Count prediction branch
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.count_dense = nn.Linear(filters, 1)
        
    def calculate_conv1d_output_length(self, length: int, kernel_size: int, 
                                     stride: int = 1, padding: int = 0, 
                                     dilation: int = 1) -> int:
        """Calculate output length for 1D convolution"""
        return int((length + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    
    def forward(self, x: torch.Tensor, verbose: bool = True) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass with shape printing
        
        Args:
            x: Input tensor of shape (batch_size, 4, sequence_length)
            verbose: Whether to print shape information
        """
        if verbose:
            print(f"\nInput shape: {x.shape}")
        self.layer_shapes['input'] = x.shape
        
        # Initial convolution
        x = self.initial_conv(x)
        if verbose:
            print(f"After initial conv: {x.shape}")
        self.layer_shapes['initial_conv'] = x.shape
        
        # Store original shape for residual connections
        original_x = x
        
        # Dilated convolutions with residual connections
        for i, conv in enumerate(self.dilated_convs, 1):
            # Apply dilated convolution
            conv_x = conv(x)
            
            # Calculate cropping for residual
            crop_size = (x.shape[-1] - conv_x.shape[-1]) // 2
            x_cropped = x[:, :, crop_size:-crop_size]
            
            # Add residual connection
            x = conv_x + x_cropped
            
            if verbose:
                print(f"After dilated conv {i} (dilation={2**i}): {x.shape}")
            self.layer_shapes[f'dilated_conv_{i}'] = x.shape
        
        # Profile prediction branch
        profile_out = self.profile_conv(x)
        if verbose:
            print(f"After profile conv: {profile_out.shape}")
        self.layer_shapes['profile_conv'] = profile_out.shape
        
        # Flatten profile output
        profile_out = profile_out.squeeze(1)  # Remove channel dimension
        if verbose:
            print(f"Profile output shape: {profile_out.shape}")
        self.layer_shapes['profile_output'] = profile_out.shape
        
        # Count prediction branch
        count_x = self.global_avg_pool(x)
        count_x = count_x.squeeze(-1)  # Remove spatial dimension
        count_out = self.count_dense(count_x)
        if verbose:
            print(f"Count output shape: {count_out.shape}")
        self.layer_shapes['count_output'] = count_out.shape
        
        return count_out, profile_out
    
    def print_layer_summary(self):
        """Print summary of all layer shapes"""
        print("\nLayer Shape Summary:")
        print("-" * 50)
        for layer_name, shape in self.layer_shapes.items():
            print(f"{layer_name:.<30} {str(shape)}")
        print("-" * 50)

# Test the network
def test_network():
    # Set parameters
    batch_size = 4
    sequence_len = 1000
    out_pred_len = 1000
    
    # Create model
    model = ChromBPNetTest(
        sequence_len=sequence_len,
        out_pred_len=out_pred_len,
        filters=64,
        n_dil_layers=8,
        conv1_kernel_size=21,
        profile_kernel_size=75
    )
    
    # Create dummy input
    x = torch.randn(batch_size, 4, sequence_len)
    
    # Forward pass
    print("\nRunning forward pass...")
    count_out, profile_out = model(x)
    
    # Print layer summary
    model.print_layer_summary()
    
    print("\nNetwork output sizes:")
    print(f"Count output: {count_out.shape}")
    print(f"Profile output: {profile_out.shape}")
    
    return model



In [2]:
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

def multinomial_nll(true_counts, logits):
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example, logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / tf.cast(tf.shape(true_counts)[0], dtype=tf.float32))

# PyTorch Implementation
class MultinomialNLLLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits, true_counts):
        # Compute log probabilities for each category
        log_probs = F.log_softmax(logits, dim=-1)
        print("PyTorch Log Probabilities:")
        print(log_probs)

        # Compute the log-likelihood manually
        likelihood = torch.sum(true_counts * log_probs, dim=-1)
        print("PyTorch Likelihoods:")
        print(likelihood)

        # Return negative mean log-likelihood
        return -likelihood.mean()

# Generate random input data
batch_size = 32
num_categories = 1000

# Random logits and true counts for testing
np.random.seed(42)
logits_np = np.random.randn(batch_size, num_categories).astype(np.float32)
true_counts_np = np.random.randint(1, 10, size=(batch_size, num_categories)).astype(np.float32)

# Convert to PyTorch tensors
logits_pt = torch.tensor(logits_np, requires_grad=True)
true_counts_pt = torch.tensor(true_counts_np)

# Convert to TensorFlow tensors
logits_tf = tf.convert_to_tensor(logits_np)
true_counts_tf = tf.convert_to_tensor(true_counts_np)

# Compute loss using PyTorch
print("=== PyTorch Computation ===")
loss_fn_pt = MultinomialNLLLoss()
loss_pt = loss_fn_pt(logits_pt, true_counts_pt).item()
print("PyTorch Loss:", loss_pt)

# Compute loss using TensorFlow
print("\n=== TensorFlow Computation ===")
loss_tf = multinomial_nll(true_counts_tf, logits_tf).numpy()
print("TensorFlow Loss:", loss_tf)

# Print difference
print("\nDifference:", abs(loss_pt - loss_tf))

AttributeError: module 'tensorflow.python.framework.type_spec' has no attribute '_NAME_TO_TYPE_SPEC'

In [10]:
true_counts_pt.shape

torch.Size([4, 3])

In [11]:
true_counts_tf.shape

TensorShape([4, 3])

In [6]:
import tensorflow as tf
import tensorflow_probability as tfp


#from https://github.com/kundajelab/basepair/blob/cda0875571066343cdf90aed031f7c51714d991a/basepair/losses.py#L87
def multinomial_nll(true_counts, logits):
    """Compute the multinomial negative log-likelihood
    Args:
      true_counts: observed count values
      logits: predicted logit values
    """
    counts_per_example = tf.reduce_sum(true_counts, axis=-1)
    dist = tfp.distributions.Multinomial(total_count=counts_per_example,
                                         logits=logits)
    return (-tf.reduce_sum(dist.log_prob(true_counts)) / 
            tf.cast(tf.shape(true_counts)[0], dtype=tf.float32))





AttributeError: module 'tensorflow.python.framework.type_spec' has no attribute '_NAME_TO_TYPE_SPEC'

In [3]:
class ImprovedMultinomialNLLLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits, true_counts):
        # Compute log probabilities for each category
        log_probs = F.log_softmax(logits, dim=-1)
        
        # Compute the log-likelihood manually
        log_likelihood = torch.sum(true_counts * log_probs, dim=-1)
        
        # Compute total counts per example
        total_counts = true_counts.sum(dim=-1)
        
        # Compute the log-gamma term for normalization (log-gamma of total_counts + 1)
        log_gamma_term = torch.lgamma(total_counts + 1)
        
        # Compute log-gamma term for each count + 1
        log_gamma_counts = torch.lgamma(true_counts + 1).sum(dim=-1)
        
        # Combine the terms to compute the final log-likelihood
        log_likelihood = log_likelihood + log_gamma_term - log_gamma_counts
        
        # Return negative mean log-likelihood
        return -torch.mean(log_likelihood)

In [4]:
import numpy as np
import torch

# Example inputs
batch_size = 4
num_categories = 3
np.random.seed(42)
logits_np = np.random.randn(batch_size, num_categories).astype(np.float32)
true_counts_np = np.random.randint(1, 10, size=(batch_size, num_categories)).astype(np.float32)


# Convert numpy arrays to PyTorch tensors
logits_torch = torch.tensor(logits_np)
true_counts_torch = torch.tensor(true_counts_np)

# Compute PyTorch loss
loss_fn_torch = ImprovedMultinomialNLLLoss()
loss_torch = loss_fn_torch(logits_torch, true_counts_torch).item()

print("Improved Multinomial NLL Loss (PyTorch):", loss_torch)


Improved Multinomial NLL Loss (PyTorch): 8.918990135192871


In [5]:
import tensorflow as tf

In [ ]:

# Convert numpy arrays to TensorFlow tensors
logits_tf = tf.convert_to_tensor(logits_np, dtype=tf.float32)
true_counts_tf = tf.convert_to_tensor(true_counts_np, dtype=tf.float32)

# Compute TensorFlow loss
loss_tf = tensorflow_multinomial_nll(true_counts_tf, logits_tf).numpy()

print("Multinomial NLL Loss (TensorFlow):", loss_tf)